##### Download / install packages if needed

In [ ]:
#Uncomment these if needed
#!conda install -c conda-forge wikipedia --yes
#!conda install -c conda-forge pandasql --yes

##### Import Packages

In [1]:
import pandas as pd
import wikipedia as wp
import numpy as np
import pandasql as pdsql
pysql = lambda q: pdsql.sqldf(q, globals())


##### Read Geo Data Into Dataframe

In [ ]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')

##### Read Postal code / Borough / Neighborhood data into Dataframe (results of previous notebook)

In [ ]:
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
# Remove Not assigned values from Borough
pysql = lambda q: pdsql.sqldf(q, globals())
df=pysql('select * from df where borough!=\'Not assigned\'')
# Boroughs with Not assigned vale for Neighborhood get the Borough name as Neighborhood
df=pysql('select Postcode, Borough, CASE when Neighbourhood=\'Not assigned\' then Borough else Neighbourhood END as Neighbourhood from df order by Postcode, Borough')
# Convert Neighborhoods into a list as opposed to separate records
# Manipulate data in a numpy array
x=df.to_numpy()
y=int(len(df.index))-1
z=0
q=z-1
while z<y:
    while q<=(y-1):
        if x[z,0]==x[q,0] and x[z,1]==x[q,1] and x[z,2]!=x[q,2]:
            W=x[q,2] + "," + x[z,2]
            x[z,2]=W
            x[q,2]=W
        q=q+1
        z=z+1
# Put data back in dataframe
df=pd.DataFrame({'Postcode': x[:, 0], 'Borough': x[:, 1], 'Neighbourhood': x[:, 2]})
#Query insures we get accurate multi-neighborhood records
df=pysql('select distinct Postcode, Borough, Neighbourhood from df where length(Neighbourhood)=(select max(length(Neighbourhood)) from df as q where df.Postcode=q.Postcode and df.Borough=q.Borough)')

##### Combine the two dataframes into one dataframe - I'm using pysql to do this

In [ ]:
df_combined=pysql('select df.Postcode, df.Borough, df.Neighbourhood, df_geo.Latitude, df_geo.Longitude from df INNER JOIN df_geo on df.Postcode=df_geo."Postal Code"')
df_combined.head(10)